In [ ]:
import itertools
import networkx as nx
import numpy as np

import network

from preprocess_data import GAMES, MAX_PENALTY_NUMBER, build_networks, Game, Venue, Situation, PowerPlay, directed_to_undirected
from datetime import date
from enum import Enum

import matplotlib.mlab as mlab
import matplotlib.pyplot as plt

from scipy import stats

In [ ]:
def regular_networks():
    for game, venue, (situation, pp) in itertools.product(
        GAMES,
        [Venue.HOME, Venue.AWAY],
        [(Situation.REGULAR, None)]
    ):
        res = build_networks(Game(game), venue=venue, situation=situation, pp=pp)
        if res is not None:
            yield network.process_graph_for_analysis(res["position_pass_network"])

In [ ]:
def power_play_networks():
    for game, venue, pp in itertools.product(
        GAMES,
        [Venue.HOME, Venue.AWAY],
        list(map(PowerPlay, range(1, MAX_PENALTY_NUMBER + 1)))
    ):
        res = build_networks(Game(game), venue=venue, situation=Situation.POWER_PLAY, pp=pp)
        if res is not None:
            yield network.process_graph_for_analysis(res["position_pass_network"])

In [ ]:
positions = ['Left Defense', 'Left Wing', 'Center', 'Right Defense', 'Right Wing']

In [ ]:
# Regular Networks
regular_centralities = dict((pos, []) for pos in positions)

fig, axs = plt.subplots(5, 1, sharex=True)
fig.set_figheight(15)
fig.set_figwidth(15)

for position, ax in zip(positions, axs):
    x = [network.centrality(nw)[position] for nw in regular_networks()]
    regular_centralities[position] = x
    n, bins, patches = ax.hist(x, facecolor='blue', alpha=0.5, weights=np.ones_like(x) / len(x))
    ax.set_title(position)
    ax.set_xlim([0, 40])
    ax.set_ylim([0, 0.5])

plt.savefig("regular.png", dpi=300)
plt.show()


In [ ]:
# Power Plays
pp_centralities = dict((pos, []) for pos in positions)

fig, axs = plt.subplots(5, 1, sharex=True)
fig.set_figheight(15)
fig.set_figwidth(15)


for position, ax in zip(positions, axs):
    # Ah, good old complicated python list comprehensions.
    x = [centr[position] 
         for nw in power_play_networks() 
         if position in (centr := network.centrality(nw)).keys()]
    pp_centralities[position] = x
    print(f"{position} occurrences: {len(x)}")
    n, bins, patches = ax.hist(x, facecolor='blue', alpha=0.5, weights=np.ones_like(x) / len(x))
    ax.set_title(position)
    ax.set_xlim([0, 40])
    ax.set_ylim([0, 0.5])

# print(f"{failed_cnt / 5} / {cnt / 5} networks failed")
plt.savefig("pp.png", dpi=300)
plt.show()


In [ ]:
for position in positions:
    # TODO: I am no statistician, but samples used in pp_centralities are not independent; 
    # so, probably something cleverer (done by a less tired person) should be done.
    print(position)
    t = stats.ttest_ind(regular_centralities[position], pp_centralities[position], equal_var=False, alternative="greater")
    print(t)